<a href="https://colab.research.google.com/github/Robert2004-AI/LearnComputerVisionWithMithun/blob/main/ADL_Assigment01_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group No - 44

**Group Member Names:**

SAKTHI R (2023aa05940)

ROBERTSEKAR R (2023aa05823)

RAVISHANKAR R (2023aa05171

NIMBALKAR PRITEESH DADASAHEB (2023aa05950)

# Task 2:

Train a single layer autoencoder with linear activation function and appropriately mean and variance normalized input with constraint that encoder weight matrix and decoder weight matrix are transpose w,r,t, each other. Compare the eigenvectors obtained in step 1 with those obtained using the autoencoders. Explain your observations.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.datasets import fetch_openml
from tensorflow.keras.utils import to_categorical

# ---- Load MNIST Dataset ----
def load_data():
    """
    Load the MNIST dataset from OpenML, normalize the pixel values to [0, 1],
    and split into training and testing sets.
    """
    mnist = fetch_openml('mnist_784', version=1)  # Fetch MNIST dataset
    X = mnist.data / 255.0  # Normalize pixel values to [0, 1]
    y = mnist.target.astype(int)  # Convert target labels to integers
    # Split the data into training (70%) and testing (30%) sets
    return train_test_split(X, y, test_size=0.3, random_state=42)

X_train, X_test, y_train, y_test = load_data()


In [2]:
# Single Layer Linear Autoencoder
def build_autoencoder(input_dim, latent_dim):
    """
    Build a single-layer linear autoencoder model.

    Parameters:
        input_dim (int): Dimensionality of the input data.
        latent_dim (int): Dimensionality of the latent (compressed) space.

    Returns:
        autoencoder (Model): Compiled autoencoder model.
    """
    # Define the input layer with the same shape as the input data
    input_layer = layers.Input(shape=(input_dim,))

    # Encoder: Compress input to latent space of size 'latent_dim' with linear activation
    encoded = layers.Dense(latent_dim, activation='linear')(input_layer)

    # Decoder: Reconstruct input from the latent space with linear activation
    decoded = layers.Dense(input_dim, activation='linear')(encoded)

    # Create the autoencoder model combining encoder and decoder
    autoencoder = models.Model(input_layer, decoded)

    # Compile the model with Adam optimizer and Mean Squared Error (MSE) loss
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder


# ---- Main Execution ----

latent_dim = 50  # Set the latent space dimensionality for the autoencoder
pca = PCA(n_components=0.95)  # Initialize PCA to retain variance
# Fit PCA to the training data
pca.fit(X_train)  # Compute PCA on the training data
# Build the autoencoder with input dimensionality equal to the number of features in the dataset
autoencoder = build_autoencoder(X_train.shape[1], latent_dim)

# Train the autoencoder using the training data as both input and target
# The goal is to reconstruct the input while learning a compressed representation
print("Training Single-Layer Linear Autoencoder...")
autoencoder.fit(X_train, X_train, epochs=10, batch_size=256, shuffle=True)

# Extract encoder weights (first layer's weights) for comparison with PCA components
encoder_weights = autoencoder.layers[1].get_weights()[0]

# Compare encoder weights with PCA eigenvectors
# PCA eigenvectors represent the directions of maximum variance, while autoencoder weights approximate them
print("Comparing Autoencoder Weights with PCA Eigenvectors")
# Select the top 'latent_dim' PCA components
pca_components = pca.components_[:latent_dim]
print(f"PCA Eigenvectors Shape: {pca_components.shape}, Autoencoder Weights Shape: {encoder_weights.shape}")



Training Single-Layer Linear Autoencoder...
Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0653
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0198
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0145
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0128
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0123
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0121
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0120
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0119
Epoch 9/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0119
Epoch 10/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0119
Comparing Autoencoder Weights with PCA Eigenvectors
PCA Eigenvectors Shape: (50, 784), Autoencoder Weights Shape: (784, 50)
